Data for sales/penetration will not be ready until after the 8th of the month.

### ***Pull Construction Funding at the beginning of the month, only once a month

December 2022 funding data - already pulled 11/30/22 - do not pull until after 12/31

In [1]:
import os
import sys
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
# Import wire center datasource from 'Get Accurate Wire Center Base' project
Sales_WC_excel = pd.read_excel(r'WireCenterBase/FINAL_wire_center_mapping_DO-NOT-ALTER.xlsx',sheet_name = 'Sheet1')

In [3]:
# clean base table
# check all states are legit
# Sales_WC_excel['STATE'].value_counts()

wc_df = Sales_WC_excel.copy() #save base table

In [4]:
wc_df = wc_df.rename(columns = {'WIRE_CENTER_ID': 'PRIMARY_WIRE_CENTER_ID',
                                'CBSA_NAME': 'CBSA',
                                'DMA_L2': 'DMA'})

In [5]:
# Import excel MG readiness data
MG_excel_df = pd.read_excel(r'120122_OpportunityScore_MultiGig_Readiness.xlsx',sheet_name = 'Datasheet')

# Import excel opportunity score data
OppScore_excel_df = pd.read_excel(r'OpportunityScoresMapping/CLLI_ZIP_XREF_WEIGHTED_SCORE.xlsx',sheet_name = 'Final')

In [6]:
# join in opportunity score
wc_ora_df = wc_df.merge(OppScore_excel_df[['CLLI','Weighted Opportunity Score']], how = 'left', left_on = 'PRIMARY_WIRE_CENTER_ID', right_on = 'CLLI')

In [7]:
wc_ora_df = wc_ora_df.rename(columns = {'DMA_NAME': 'DMA'})

In [8]:
# clean data
wc_ora_df = wc_ora_df[['STATE','DMA','CBSA','Weighted Opportunity Score','PRIMARY_WIRE_CENTER_ID']].rename(
    columns = {
        'Weighted Opportunity Score':'OPPORTUNITY_SCORE'})

In [9]:
# clean mg readiness data - getting core status and monthly planned build - month part needs to be hardcoded manually
MG_excel_df = MG_excel_df[['PRIMARY_WIRE_CENTER_ID','Core Status']].rename(
    columns = {
        'Core Status': 'CORE_READINESS',
})

In [10]:
# join in planned xgs build/core status

wc_ora_df = wc_ora_df.merge(MG_excel_df, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

In [11]:
# check data
wc_ora_df.head()

,STATE,DMA,CBSA,OPPORTUNITY_SCORE,PRIMARY_WIRE_CENTER_ID,CORE_READINESS
0,SD,SIOUX FALLS (MITCHELL) SD,"ABERDEEN, SD",6.0,ABRDSDCO,NaN
1,WA,SEATTLE-TACOMA WA,"ABERDEEN, WA",8.0,ABRDWA01,NaN
2,IA,DES MOINES-AMES IA,NON-CBSA,NaN,ACKLIACO,NaN
3,OR,EUGENE OR,"CORVALLIS, OR",5.0,ADAROR21,NaN
4,IA,DES MOINES-AMES IA,"DES MOINES-WEST DES MOINES, IA",14.0,ADELIACO,NaN


In [12]:
#Import Funding data
Construction_Funding_df = pd.read_excel(r'FundingDetailsExport/2023_GPON_Details_Export_Table.xlsx',sheet_name = 'Export')

In [13]:
Construction_Funding_df["PHASE_STAR_ID_20"].value_counts()

Low Confidence - High Risk         48
Medium Confidence - Medium Risk    28
High Confidence - Low Risk         13
High Confidence                    11
LOW CONFIDENCE/HIGH RISK           10
Low Confidence                      7
Med Probability/Med Risk            2
Medium Confidence                   2
Low Probability/High Risk           1
MEDIUM CONFIDENCE/MEDIUM RISK       1
LEGACY LEEP ONLY                    1
6-HIGH CONFIDENCE - LOW RISK        1
Med                                 1
Name: PHASE_STAR_ID_20, dtype: int64

In [14]:
Construction_Funding_df_high = Construction_Funding_df[(Construction_Funding_df["PHASE_STAR_ID_20"]=="High Confidence - Low Risk") | 
                        (Construction_Funding_df["PHASE_STAR_ID_20"]=="High Confidence") | 
                        (Construction_Funding_df["PHASE_STAR_ID_20"]=="6-HIGH CONFIDENCE - LOW RISK")
                    ]

# Construction_Funding_df_medium = Construction_Funding_df[(Construction_Funding_df["PHASE_STAR_ID_20"]=="Medium Confidence - Medium Risk") | 
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]=="Med Probability/Med Risk") | 
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]=="Medium Confidence") |
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]=="MEDIUM CONFIDENCE/MEDIUM RISK") |
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]=="Med")
#                     ]

# Construction_Funding_df_low = Construction_Funding_df[
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]!="High Confidence - Low Risk") &
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]!="High Confidence") & 
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]!="6-HIGH CONFIDENCE - LOW RISK") &
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]!="Medium Confidence - Medium Risk") &
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]!="Med Probability/Med Risk") & 
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]!="Medium Confidence") &
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]!="MEDIUM CONFIDENCE/MEDIUM RISK") &
#                         (Construction_Funding_df["PHASE_STAR_ID_20"]!="Med")
#                     ]

In [15]:
Construction_Funding_df_high['PHASE_ECD_LOGIC'] = Construction_Funding_df_high['PHASE_ECD_LOGIC'].dt.to_period('M')
Construction_Funding_df_high['PHASE_ECD_LOGIC'] = Construction_Funding_df_high['PHASE_ECD_LOGIC'].astype(str)
Construction_pivot = Construction_Funding_df_high.groupby(by = ["PRIMARY_WIRE_CENTER_ID","PHASE_ECD_LOGIC"], as_index = False).sum()
Construction_pivot = Construction_pivot.pivot(index='PRIMARY_WIRE_CENTER_ID', columns='PHASE_ECD_LOGIC')['PHASE_NUM_OF_LUS'].reset_index()
Construction_pivot = Construction_pivot.fillna(0)

C:\Windows\Temp\ipykernel_4904\4056556507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Construction_Funding_df_high['PHASE_ECD_LOGIC'] = Construction_Funding_df_high['PHASE_ECD_LOGIC'].dt.to_period('M')
C:\Windows\Temp\ipykernel_4904\4056556507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Construction_Funding_df_high['PHASE_ECD_LOGIC'] = Construction_Funding_df_high['PHASE_ECD_LOGIC'].astype(str)


In [16]:
# rename columns
Construction_Funding_df_high = Construction_pivot.copy()
Construction_Funding_df_high = Construction_Funding_df_high.groupby(
    by = ["PRIMARY_WIRE_CENTER_ID"], as_index = False).sum().rename(
    columns = {
        '2022-09': 'SEP-22_GPON_CONSTRUCTION_FUNDED_high',
        '2022-10': 'OCT-22_GPON_CONSTRUCTION_FUNDED_high',
        '2022-11': 'NOV-22_GPON_CONSTRUCTION_FUNDED_high',
        '2022-12': 'DEC-22_GPON_CONSTRUCTION_FUNDED_high',
        '2023-01': 'JAN-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-02': 'FEB-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-03': 'MAR-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-04': 'APR-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-05': 'MAY-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-06': 'JUN-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-07': 'JUL-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-08': 'AUG-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-09': 'SEP-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-10': 'OCT-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-11': 'NOV-23_GPON_CONSTRUCTION_FUNDED_high',
        '2023-12': 'DEC-23_GPON_CONSTRUCTION_FUNDED_high',
})

In [17]:
# join in construction funding to base table

wc_ora_df = wc_ora_df.merge(Construction_Funding_df_high, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

In [18]:
wc_ora_df.head()

,STATE,DMA,CBSA,OPPORTUNITY_SCORE,PRIMARY_WIRE_CENTER_ID,CORE_READINESS,JAN-23_GPON_CONSTRUCTION_FUNDED_high,FEB-23_GPON_CONSTRUCTION_FUNDED_high,MAR-23_GPON_CONSTRUCTION_FUNDED_high,MAY-23_GPON_CONSTRUCTION_FUNDED_high,JUL-23_GPON_CONSTRUCTION_FUNDED_high,NOV-23_GPON_CONSTRUCTION_FUNDED_high
0,SD,SIOUX FALLS (MITCHELL) SD,"ABERDEEN, SD",6.0,ABRDSDCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WA,SEATTLE-TACOMA WA,"ABERDEEN, WA",8.0,ABRDWA01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IA,DES MOINES-AMES IA,NON-CBSA,NaN,ACKLIACO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OR,EUGENE OR,"CORVALLIS, OR",5.0,ADAROR21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IA,DES MOINES-AMES IA,"DES MOINES-WEST DES MOINES, IA",14.0,ADELIACO,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
import time

# export
TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_GPON_high_confidence.xlsx"

with pd.ExcelWriter(excelfilename) as writer:  
    wc_ora_df.to_excel(writer, sheet_name='Cumulative Data', index = False)